In [188]:
best_Action = []

## 강화학습 모델 생성

In [2]:
from scipy.io import loadmat
import numpy as np
np.set_printoptions(formatter={'float': '{:0.5f}'.format})

import pandas as pd

# .mat 파일 로드
data = loadmat('pvLoadPriceData_test.mat')

# 데이터 추출 예시
# MATLAB 파일 내에 있는 변수 이름을 정확히 알아야 합니다. time, cloudyDay, clearDay, loadData, costData
time = data['time'].flatten()
solarData = data['clearDay'].flatten() * 10e3 
loadData = data['loadData'][:, 2].flatten() 
costData = data['costData'].flatten() 

print("자료형:", type(time), "차원" , time.shape, )
print("자료형:", type(solarData), "차원" , solarData.shape, )
print("자료형:", type(loadData), "차원" , loadData.shape, )
print("자료형:", type(costData), "차원" , costData.shape, )

solarData = solarData.tolist()
loadData = loadData.tolist()
costData = costData.tolist()

자료형: <class 'numpy.ndarray'> 차원 (228,)
자료형: <class 'numpy.ndarray'> 차원 (228,)
자료형: <class 'numpy.ndarray'> 차원 (228,)
자료형: <class 'numpy.ndarray'> 차원 (228,)


In [3]:
print(solarData)
print(loadData)
print(costData)

[35000.0, 35100.0, 34900.0, 34500.0, 34500.0, 34700.0, 34700.0, 34900.0, 34500.0, 34500.0, 34700.0, 34700.0, 34900.0, 34700.0, 34900.0, 35300.0, 35100.0, 36000.0, 36500.0, 36000.0, 35900.0, 37500.0, 37300.0, 38400.0, 40199.99999999999, 47300.00000000001, 52800.0, 60000.0, 56600.0, 46400.0, 51900.00000000001, 50599.99999999999, 55100.0, 58900.0, 55900.0, 53300.0, 49900.0, 55599.99999999999, 52400.0, 54500.0, 57200.0, 62800.0, 69200.0, 48500.0, 50500.0, 84300.0, 88100.0, 88200.0, 85900.0, 87600.0, 90000.0, 81900.0, 76700.0, 88800.00000000001, 93400.0, 109500.0, 120700.0, 116800.0, 106100.0, 128400.0, 135300.0, 155500.0, 141300.0, 123500.0, 110000.0, 128600.0, 155400.0, 188100.0, 262100.0, 210600.0, 160500.0, 166700.00000000003, 187200.0, 215900.0, 219100.0, 295200.0, 279900.0, 305100.0, 207900.0, 303300.0, 272500.0, 290100.0, 313400.0, 372400.0, 340500.0, 278100.0, 382299.99999999994, 416700.0, 422000.0, 373300.0, 368000.0, 422500.0, 275700.0, 291400.0, 389900.0, 353600.0, 364799.9999999

In [12]:
solarData = pd.read_csv('/Users/happy/Desktop/캡스톤/RL/Data/Solar_Generation_Data.csv')
loadData = pd.read_csv('/Users/happy/Desktop/캡스톤/RL/Data/Power_Consumption_Data.csv')
costData = pd.read_csv('/Users/happy/Desktop/캡스톤/RL/Data/costData.csv')
costData = costData.loc[::6,].dropna().reset_index(drop=True)

In [14]:
solarData = solarData['Generation (kW)'].tolist()
loadData = loadData['Consumption (kW)'].tolist()
costData = costData["summer"].to_list()

In [15]:
print(solarData)
print(loadData)
print(costData)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.02, 0.04, 0.11, 0.25, 0.56, 1.14, 2.2, 3.98, 6.77, 10.81, 16.23, 22.89, 30.33, 37.74, 44.12, 48.46, 50.0, 48.46, 44.12, 37.74, 30.33, 22.89, 16.23, 10.81, 6.77, 3.98, 2.2, 1.14, 0.56, 0.25, 0.11, 0.04, 0.02, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.03, 0.15, 0.55, 1.64, 4.04, 8.13, 13.41, 18.1, 20.0, 18.1, 13.41, 8.13, 4.04, 1.64, 0.55, 0.15, 0.03, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.05, 0.22, 0.82, 2.46, 6.06, 12.2, 20.11, 27.15, 30.0, 27.15, 20.11, 12.2, 6.06, 2.46, 0.82, 0.22, 0.05, 0.01]
[76.5, 76.5, 76.5, 76.5, 76.5, 76.5, 76.5, 76.5, 76.5, 76.5, 76.5, 76.5, 76.5, 76.5, 76.5, 76.5, 121.2, 121.2, 121.2, 121.2, 121.2, 121.2, 187.1, 187.1, 121.2, 121.2, 187.1, 187.1, 187.1, 187.1, 187.1, 187.1, 187.1, 187.1, 187.1, 187.1, 121.2, 121.2, 121.2, 121.2, 121.2, 121.2, 121.2, 121.2, 121.2, 121.2, 121.2, 121.2]


In [38]:
import numpy as np
import gym
from gym import spaces

# 전력망 환경 클래스 정의
class PowerGridEnv(gym.Env):
    def __init__(self):
        self.solar_data = solarData
        self.load_data = loadData
        self.price_data = costData

        BattCap = 2500  # 에너지 저장 장치 용량 [kWh]
        self.battEnergy = 3.6e6 * BattCap # 총 배터리 용량
        self.battery = ((0.5 * self.battEnergy)/self.battEnergy) * 100  # 초기 배터리 상태
        self.batteryMin = ((0.2 * self.battEnergy)/self.battEnergy) * 100
        self.batteryMax = ((0.8 * self.battEnergy)/self.battEnergy) * 100
        self.total_grid_cost = 0
        self.state = None

        # 배터리 상태
        self.observation_space = spaces.Box(low=np.array([0]), high=np.array([100]))
        
        # 액션 종류
        self.action_space = spaces.Discrete(7)
        self.data_pointer = 0
        

    def reset(self):
        self.data_pointer = 0
        self.battery = ((0.5 * self.battEnergy)/self.battEnergy) * 100
        self.total_grid_cost = 0
        self.state = [self.solar_data[0], self.load_data[0], self.price_data[0], self.battery]
        return np.array(self.state)
    
    def render(self, mode='human'):
        # 가정: self.state는 환경의 현재 상태를 나타냅니다.
        # 간단한 텍스트 출력으로 상태를 표현합니다.
        print(f"Current state: {self.state}")
        
    def step(self, action):
        reward = 0
       
        tPV = self.solar_data[self.data_pointer]    # 스탭 당 태양광 발전량
        tLoad = self.load_data[self.data_pointer]   # 스탭 당 부하량
        tPrice = self.price_data[self.data_pointer] # 스탭 당 전력 가격

        tP2B = 0
        tB2P = 0
        tNeedPower = 0

        # 액션에 따른 배터리 및 그리드 상호작용
        if action == 0 : # 태양광 O ESS X 그리드 X
            if tPV >= tLoad :
                tSurplusPower = tPV - tLoad # 잉여 전력 계산 -> 잉여 전력이 있다면 배터리에 저장
                if tSurplusPower > 0 : 
                    # 잉여 전력량
                    tP2B = (tSurplusPower / self.battEnergy) * 100

                    # 배터리의 충전 가능(%)
                    tPossible_Batt = self.batteryMax - self.battery

                    # 배터리에 충전할 전력 계산
                    tPowerChg = min(tP2B, tPossible_Batt) # 잉여 전력이랑 배터리 충전 가능량을 비교해서 더 작은 값을 충전
                    tSurplusChg = tP2B - tPowerChg # 충전하고 남은 전력량
                    
                    # 배터리를 충전했을 때 20~80% 일 때
                    if ((self.battery + tPowerChg) > self.batteryMin) and ((self.battery + tPowerChg) < self.batteryMax) :
                        self.battery += tPowerChg
                    # 배터리를 충전했을 때 80% 이상일 때
                    else :
                        self.total_grid_cost -= ((tP2B * self.battEnergy)/100) * tPrice

            else : # tLoad > tPV
                # 태양광 발전량으로 커버한 뒤 필요한 전력을 계산
                tNeedPower = tLoad - tPV
                tB2P = (tNeedPower/self.battEnergy)*100

                # 배터리의 방전 가능(%)
                tPossible_Batt = self.battery - self.batteryMin

                # 배터리에 방전할 전력 계산
                tPowerDisChg = min(tB2P, tPossible_Batt) # 필요한 전력이랑 배터리 뱡젼 가능량을 비교해서 더 작은 값을 방전
                tSurplusChg = tB2P - tPowerDisChg # 충전하고 남은 전력량
                
                # 배터리를 방전했을 때 20~80% 일 때
                if ((self.battery - tPowerDisChg) > self.batteryMin) and ((self.battery - tPowerDisChg) < self.batteryMax) :
                    self.battery -= tPowerDisChg
                # 배터리를 방전했을 때 20% 이하일 때
                else :
                    self.total_grid_cost += ((tB2P * self.battEnergy)/100) * tPrice


        elif action == 1 : # 태양광 O ESS O 그리드 X
            if tLoad > tPV :
                # 태양광 발전량으로 커버한 뒤 필요한 전력을 계산
                tNeedPower = tLoad - tPV
                tB2P = (tNeedPower/self.battEnergy)*100

                # 배터리의 방전 가능(%)
                tPossible_Batt = self.battery - self.batteryMin

                # 배터리에 방전할 전력 계산
                tPowerDisChg = min(tB2P, tPossible_Batt) # 필요한 전력이랑 배터리 뱡젼 가능량을 비교해서 더 작은 값을 방전
                tSurplusChg = tB2P - tPowerDisChg # 충전하고 남은 전력량
                
                # 배터리를 방전했을 때 20~80% 일 때
                if ((self.battery - tPowerDisChg) > self.batteryMin) and ((self.battery - tPowerDisChg) < self.batteryMax) :
                    self.battery -= tPowerDisChg
                # 배터리를 방전했을 때 20% 이하일 때
                else :
                    self.total_grid_cost += ((tB2P * self.battEnergy)/100) * tPrice


            else : # tLoad < tPV 
                tSurplusPower = tPV - tLoad # 잉여 전력 계산 -> 잉여 전력이 있다면 배터리에 저장
                if tSurplusPower > 0 : 
                    # 잉여 전력량
                    tP2B = (tSurplusPower / self.battEnergy) * 100

                    # 배터리의 충전 가능(%)
                    tPossible_Batt = self.batteryMax - self.battery

                    # 배터리에 충전할 전력 계산
                    tPowerChg = min(tP2B, tPossible_Batt) # 잉여 전력이랑 배터리 충전 가능량을 비교해서 더 작은 값을 충전
                    tSurplusChg = tP2B - tPowerChg # 충전하고 남은 전력량
                    
                    # 배터리를 충전했을 때 20~80% 일 때
                    if ((self.battery + tPowerChg) > self.batteryMin) and ((self.battery + tPowerChg) < self.batteryMax) :
                        self.battery += tPowerChg
                    # 배터리를 충전했을 때 80% 이상일 때
                    else :
                        self.total_grid_cost -= ((tP2B * self.battEnergy)/100) * tPrice

    
        elif action == 2 : # 태양광 O ESS X 그리드 O
            if tLoad > tPV :
                tNeedPower = tLoad - tPV # 태양광 발전량으로 커버한 뒤 필요한 전력을 계산
                if tNeedPower > 0 : # 전력이 더 필요할 경우
                    self.total_grid_cost += (tNeedPower * tPrice)

            else : # tLoad < tPV 
                tSurplusPower = tPV - tLoad # 잉여 전력 계산 -> 잉여 전력이 있다면 배터리에 저장
                if tSurplusPower > 0 : 
                    # 잉여 전력량
                    tP2B = (tSurplusPower / self.battEnergy) * 100

                    # 배터리의 충전 가능(%)
                    tPossible_Batt = self.batteryMax - self.battery

                    # 배터리에 충전할 전력 계산
                    tPowerChg = min(tP2B, tPossible_Batt) # 잉여 전력이랑 배터리 충전 가능량을 비교해서 더 작은 값을 충전
                    tSurplusChg = tP2B - tPowerChg # 충전하고 남은 전력량
                    
                    # 배터리를 충전했을 때 20~80% 일 때
                    if ((self.battery + tPowerChg) > self.batteryMin) and ((self.battery + tPowerChg) < self.batteryMax) :
                        self.battery += tPowerChg
                    # 배터리를 충전했을 때 80% 이상일 때
                    else :
                        self.total_grid_cost -= ((tP2B * self.battEnergy)/100) * tPrice 


        elif action == 3 : # 태양광 O ESS O 그리드 O
            if tLoad > tPV :
                # 태양광 발전량으로 커버한 뒤 필요한 전력을 계산
                tNeedPower = tLoad - tPV
                tB2P = (tNeedPower/self.battEnergy)*100

                # 배터리의 방전 가능(%)
                tPossible_Batt = self.battery - self.batteryMin

                # 배터리에 방전할 전력 계산
                tPowerDisChg = min(tB2P, tPossible_Batt) # 필요한 전력이랑 배터리 뱡젼 가능량을 비교해서 더 작은 값을 방전
                tSurplusChg = tB2P - tPowerDisChg # 충전하고 남은 전력량
                
                # 배터리를 방전했을 때 20~80% 일 때
                if ((self.battery - tPowerDisChg) > self.batteryMin) and ((self.battery - tPowerDisChg) < self.batteryMax) :
                    self.battery -= tPowerDisChg
                # 배터리를 방전했을 때 20% 이하일 때
                else :
                    self.battery -= tPowerDisChg
                    self.total_grid_cost += ((tSurplusChg * self.battEnergy)/100) * tPrice
                
            else : # tLoad < tPV 
                tSurplusPower = tPV - tLoad # 잉여 전력 계산 -> 잉여 전력이 있다면 배터리에 저장
                if tSurplusPower > 0 : 
                    # 잉여 전력량
                    tP2B = (tSurplusPower / self.battEnergy) * 100

                    # 배터리의 충전 가능(%)
                    tPossible_Batt = self.batteryMax - self.battery

                    # 배터리에 충전할 전력 계산
                    tPowerChg = min(tP2B, tPossible_Batt) # 잉여 전력이랑 배터리 충전 가능량을 비교해서 더 작은 값을 충전
                    tSurplusChg = tP2B - tPowerChg # 충전하고 남은 전력량
                    
                    # 배터리를 충전했을 때 20~80% 일 때
                    if ((self.battery + tPowerChg) > self.batteryMin) and ((self.battery + tPowerChg) < self.batteryMax) :
                        self.battery += tPowerChg
                    # 배터리를 충전했을 때 80% 이상일 때
                    else :
                        self.total_grid_cost -= ((tP2B * self.battEnergy)/100) * tPrice 
                            

        elif action == 4 : # 태양광 X ESS O 그리드 X
            if tLoad > tPV :
                tB2P = (tLoad/self.battEnergy) * 100
                # 배터리의 방전 가능(%)
                tPossible_Batt = self.battery - self.batteryMin

                # 배터리에 방전할 전력 계산
                tPowerDisChg = min(tB2P, tPossible_Batt) # 필요한 전력이랑 배터리 뱡젼 가능량을 비교해서 더 작은 값을 방전
                tSurplusChg = tB2P - tPowerDisChg # 충전하고 남은 전력량

                # 배터리를 방전했을 때 20~80% 일 때
                if ((self.battery - tPowerDisChg) > self.batteryMin) and ((self.battery - tPowerDisChg) < self.batteryMax) :
                    self.battery -= tPowerDisChg
                # 배터리를 방전했을 때 20% 이하일 때
                else :
                    self.total_grid_cost += ((tB2P * self.battEnergy)/100) * tPrice
            
            else : # tLoad < tPV 
                tSurplusPower = tPV - tLoad # 잉여 전력 계산 -> 잉여 전력이 있다면 배터리에 저장
                if tSurplusPower > 0 : 
                    # 잉여 전력량
                    tP2B = (tSurplusPower / self.battEnergy) * 100

                    # 배터리의 충전 가능(%)
                    tPossible_Batt = self.batteryMax - self.battery

                    # 배터리에 충전할 전력 계산
                    tPowerChg = min(tP2B, tPossible_Batt) # 잉여 전력이랑 배터리 충전 가능량을 비교해서 더 작은 값을 충전
                    tSurplusChg = tP2B - tPowerChg # 충전하고 남은 전력량
                    
                    # 배터리를 충전했을 때 20~80% 일 때
                    if ((self.battery + tPowerChg) > self.batteryMin) and ((self.battery + tPowerChg) < self.batteryMax) :
                        self.battery += tPowerChg
                    # 배터리를 충전했을 때 80% 이상일 때
                    else :
                        self.total_grid_cost -= ((tP2B * self.battEnergy)/100) * tPrice 

        elif action == 5 : # 태양광 X ESS X 그리드 O
            self.total_grid_cost += tLoad * tPrice

        elif action == 6 : # 태양광 X ESS O 그리드 O
            if tLoad > tPV :
                tB2P = (tLoad/self.battEnergy) * 100
                # 배터리의 방전 가능(%)
                tPossible_Batt = self.battery - self.batteryMin

                # 배터리에 방전할 전력 계산
                tPowerDisChg = min(tB2P, tPossible_Batt) # 필요한 전력이랑 배터리 뱡젼 가능량을 비교해서 더 작은 값을 방전
                tSurplusChg = tB2P - tPowerDisChg # 충전하고 남은 전력량

                # 배터리를 방전했을 때 20~80% 일 때
                if ((self.battery - tPowerDisChg) > self.batteryMin) and ((self.battery - tPowerDisChg) < self.batteryMax) :
                    self.battery -= tPowerDisChg
                # 배터리를 방전했을 때 20% 이하일 때
                else :
                    self.total_grid_cost += ((tB2P * self.battEnergy)/100) * tPrice
            
            else : # tLoad < tPV 
                tSurplusPower = tPV - tLoad # 잉여 전력 계산 -> 잉여 전력이 있다면 배터리에 저장
                if tSurplusPower > 0 : 
                    # 잉여 전력량
                    tP2B = (tSurplusPower / self.battEnergy) * 100

                    # 배터리의 충전 가능(%)
                    tPossible_Batt = self.batteryMax - self.battery

                    # 배터리에 충전할 전력 계산
                    tPowerChg = min(tP2B, tPossible_Batt) # 잉여 전력이랑 배터리 충전 가능량을 비교해서 더 작은 값을 충전
                    tSurplusChg = tP2B - tPowerChg # 충전하고 남은 전력량
                    
                    # 배터리를 충전했을 때 20~80% 일 때
                    if ((self.battery + tPowerChg) > self.batteryMin) and ((self.battery + tPowerChg) < self.batteryMax) :
                        self.battery += tPowerChg
                    # 배터리를 충전했을 때 80% 이상일 때
                    else :
                        self.total_grid_cost -= ((tP2B * self.battEnergy)/100) * tPrice 


        # 데이터 포인터 갱신
        if self.data_pointer >= len(self.solar_data) - 1:
            done = True
        else:
            done = False
            self.data_pointer += 1

        reward = 1 if done else 0
        # self.state = [tPV, tLoad, tPrice, self.battery, tP2B, tB2P, action, self.total_grid_cost* 10e-6]
        self.state = [tPV, tLoad, (self.battery-tP2B+tB2P), tP2B, -tB2P, action, self.total_grid_cost* 10e-6]
        return np.array(self.state), reward, done, {"cost" : self.total_grid_cost}
    
    
    


# 환경 인스턴스 생성 및 초기화
env = PowerGridEnv()
state = env.reset()
done = False

print("       PV        Load      SOC(%)   충전량   방전량     액션   누적 비용")

while not done:
    action = env.action_space.sample()  # 무작위 액션
    state, reward, done, info = env.step(action)
    
    print("", state, "보상", reward)


       PV        Load      SOC(%)   충전량   방전량     액션   누적 비용
 [0.00000 0.00000 50.00000 0.00000 0.00000 2.00000 0.00000] 보상 0
 [0.00000 0.00000 50.00000 0.00000 0.00000 1.00000 0.00000] 보상 0
 [0.00000 0.00000 50.00000 0.00000 0.00000 1.00000 0.00000] 보상 0
 [0.00000 0.00000 50.00000 0.00000 0.00000 2.00000 0.00000] 보상 0
 [0.00000 0.00000 50.00000 0.00000 0.00000 5.00000 0.00000] 보상 0
 [0.00000 0.01000 50.00000 0.00000 0.00000 2.00000 0.00001] 보상 0
 [0.00000 0.03000 50.00000 0.00000 0.00000 5.00000 0.00003] 보상 0
 [0.01000 0.15000 50.00000 0.00000 0.00000 5.00000 0.00015] 보상 0
 [0.02000 0.55000 50.00000 0.00000 0.00000 2.00000 0.00055] 보상 0
 [0.04000 1.64000 50.00000 0.00000 -0.00000 6.00000 0.00055] 보상 0
 [0.11000 4.04000 50.00000 0.00000 0.00000 2.00000 0.00356] 보상 0
 [0.25000 8.13000 50.00000 0.00000 0.00000 5.00000 0.00978] 보상 0
 [0.56000 13.41000 50.00000 0.00000 0.00000 5.00000 0.02004] 보상 0
 [1.14000 18.10000 50.00000 0.00000 -0.00000 4.00000 0.02004] 보상 0
 [2.20000 20.00000 50.000

In [197]:
# 환경 생성
env = PowerGridEnv()

# 시뮬레이션할 에피소드 수 설정
episodes = 100000
min_cost = float('inf')  # 최소 비용을 기록하기 위한 변수
best_state = None  # 최적의 상태를 저장하기 위한 변수
best_action_sequence = []  # 최적 액션 시퀀스 저장

# 에피소드 반복 실행
for episode in range(1, episodes + 1):
    state = env.reset()  # 환경 초기화
    done = False 
    total_reward = 0
    action_sequence = []  # 현재 에피소드의 액션 시퀀스

    while not done:
        action = env.action_space.sample()  # 무작위 액션 선택
        action_sequence.append(action)  # 액션 시퀀스에 추가
        next_state, reward, done, info = env.step(action)  # 액션 적용    
        total_reward += reward

        # 비용 최소화 조건 검사
        if done and info['cost'] < min_cost:
            min_cost = info['cost']
            best_state = state  # 최적 상태 갱신
            best_action_sequence = action_sequence  # 최적 액션 시퀀스 갱신

    # 에피소드 결과 출력
    if episode % 1000 == 0:  # 진행 상황을 1000 에피소드마다 출력
        print(f'Episode: {episode}, Total Reward: {total_reward}, Minimum Cost: { min_cost * 10e-6}')
    

# 최적 상태와 그때의 비용 출력
print(f'Best State: {best_state}, Minimum Cost Achieved: {min_cost * 10e-6}')


/opt/miniconda3/envs/DQN/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Episode: 1000, Total Reward: 1, Minimum Cost: 7187.39778988273
Episode: 2000, Total Reward: 1, Minimum Cost: 6361.722721454421
Episode: 3000, Total Reward: 1, Minimum Cost: 6361.722721454421
Episode: 4000, Total Reward: 1, Minimum Cost: 6361.722721454421
Episode: 5000, Total Reward: 1, Minimum Cost: 6361.722721454421
Episode: 6000, Total Reward: 1, Minimum Cost: 6361.722721454421
Episode: 7000, Total Reward: 1, Minimum Cost: 6264.7238279980775
Episode: 8000, Total Reward: 1, Minimum Cost: 6264.7238279980775
Episode: 9000, Total Reward: 1, Minimum Cost: 6225.167839750378
Episode: 10000, Total Reward: 1, Minimum Cost: 6225.167839750378
Episode: 11000, Total Reward: 1, Minimum Cost: 6225.167839750378
Episode: 12000, Total Reward: 1, Minimum Cost: 6225.167839750378
Episode: 13000, Total Reward: 1, Minimum Cost: 6037.396192661791
Episode: 14000, Total Reward: 1, Minimum Cost: 5053.1605195913835
Episode: 15000, Total Reward: 1, Minimum Cost: 5053.1605195913835
Episode: 16000, Total Reward: 1

In [200]:
print(best_action_sequence)

[6, 4, 2, 1, 6, 5, 3, 4, 3, 2, 4, 6, 0, 6, 0, 3, 6, 3, 3, 5, 1, 5, 4, 2, 1, 2, 4, 4, 5, 2, 3, 2, 1, 5, 6, 4, 6, 1, 2, 0, 2, 4, 2, 6, 1, 2, 6, 3, 1, 3, 5, 1, 4, 3, 0, 2, 3, 5, 2, 3, 3, 3, 1, 3, 1, 4, 5, 4, 3, 4, 1, 1, 4, 6, 6, 5, 4, 2, 1, 0, 1, 1, 1, 4, 4, 0, 1, 4, 1, 5, 6, 1, 1, 1, 4, 2, 6, 3, 2, 5, 5, 4, 0, 1, 3, 0, 6, 5, 1, 1, 2, 4, 1, 3, 1, 0, 3, 3, 0, 3, 0, 1, 1, 6, 6, 4, 1, 4, 6, 1, 0, 3, 6, 0, 3, 3, 0, 2, 6, 0, 4, 0, 4, 3, 4, 4, 0, 1, 0, 1, 1, 2, 0, 1, 3, 0, 4, 4, 0, 4, 4, 6, 1, 2, 4, 0, 4, 0, 1, 1, 3, 2, 3, 3, 4, 6, 1, 3, 0, 0, 1, 3, 6, 1, 6, 0, 4, 0, 0, 0, 0, 2, 4, 2, 2, 6, 0, 1, 4, 6, 1, 0, 5, 4, 4, 0, 6, 3, 3, 1, 2, 1, 0, 3, 2, 1, 6, 4, 1, 1, 3, 6, 0, 6, 1, 5, 2, 4]


## Q-table 생성

### 연습

In [201]:
import numpy as np
import random
import gym
from gym import spaces

# 환경 정의
class CustomEnv(gym.Env):
    metadata = {'render.modes': ['console']}

    def __init__(self):
        super(CustomEnv, self).__init__()
        self.action_space = spaces.Discrete(4)  # 0: 위, 1: 아래, 2: 좌, 3: 우
        self.observation_space = spaces.Discrete(25)  # 5x5 그리드
        self.state = None
        self.grid = np.zeros((5, 5))
        self.grid[4, 4] = 1  # 목표 위치

    def reset(self):
        self.state = (0, 0)
        return self.state_to_index(self.state)

    def step(self, action):
        x, y = self.state
        if action == 0 and x > 0:
            x -= 1
        elif action == 1 and x < 4:
            x += 1
        elif action == 2 and y > 0:
            y -= 1
        elif action == 3 and y < 4:
            y += 1

        self.state = (x, y)
        done = x == 4 and y == 4
        reward = 1 if done else 0
        return self.state_to_index(self.state), reward, done, {}

    def render(self, mode='console'):
        grid = self.grid.copy()
        grid[self.state] = 2
        for row in grid:
            print(" ".join(str(int(x)) for x in row))
        print()

    def state_to_index(self, state):
        return state[0] * 5 + state[1]  # 좌표를 선형 인덱스로 변환

# Q-테이블 초기화 및 파라미터 설정
q_table = np.zeros((25, 4))
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# 환경 인스턴스 생성 및 학습
env = CustomEnv()
for i in range(1000):
    state = env.reset()
    done = False
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])

        next_state, reward, done, _ = env.step(action)
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value
        state = next_state

# 환경 테스트
state = env.reset()
env.render()
done = False
while not done:
    action = np.argmax(q_table[state])
    state, reward, done, _ = env.step(action)
    env.render()
    print(reward)


2 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 1

0 2 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 1

0
0 0 2 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 1

0
0 0 0 2 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 1

0
0 0 0 0 2
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 1

0
0 0 0 0 0
0 0 0 0 2
0 0 0 0 0
0 0 0 0 0
0 0 0 0 1

0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 2
0 0 0 0 0
0 0 0 0 1

0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 2
0 0 0 0 1

0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 2

1
